# GRAVEYARD

z_to_xy_ratio = bioim_image.meta["scale"][0]/bioim_image.meta["scale"][1]

gaussian_smoothing_sigma = (1.34,  1.34*z_to_xy_ratio, 1.34*z_to_xy_ratio)
gaussian_smoothing_truncate_range=4.0

gaussian_smoothing_sigma

In [ ]:



total_florescence = img_data[1:-1].astype( np.double ).sum(axis=0)
print(total_florescence.shape)
max_proj = ndi.maximum_filter(total_florescence,size = (3,1,1))


In [ ]:
from infer_subc.workflow import InferSubC2dWorkflowEngine

In [ ]:
from aicssegmentation.workflow import (
                                                SegmenterFunction, 
                                                FunctionParameter, 
                                                WidgetType, 
                                                WorkflowStep, 
                                                WorkflowStepCategory,
                                                Workflow,
                                                BatchWorkflow, 
                                                WorkflowDefinition,
                                                PrebuiltWorkflowDefinition,
                                                WorkflowEngine
                                                )
from aicssegmentation.workflow.workflow_config import WorkflowConfig
#from .segmenter_function import SegmenterFunction, FunctionParameter, WidgetType  # noqa F401
#from .workflow_step import WorkflowStep, WorkflowStepCategory  # noqa F401
#from .workflow import Workflow  # noqa F401
#from .batch_workflow import BatchWorkflow  # noqa F401
#from .workflow_definition import WorkflowDefinition, PrebuiltWorkflowDefinition  # noqa F401
#from .workflow_engine import  WorkflowEngine  # noqa F401

import infer_subc

Try to sub-class WorkflowConfig, Directories, etc. to get configurations from infer_subc.organelles


PreBuiltWorkflowDefinition also reference Directories, but we don't want to use it anyway...

In [ ]:
import json

class InferSubC2dDirectories:
    """
    Provides safe paths to common infer-subc module directories
    """

    _module_base_dir = Path(infer_subc.__file__).parent

    @classmethod
    def get_assets_dir(cls) -> Path:
        """
        Path to the assets directory
        """
        return cls._module_base_dir / "assets"

    @classmethod
    def get_structure_config_dir(cls) -> Path:
        """
        Path to the structure json config directory
        """
        return cls._module_base_dir / "organelles_config"



class InferSubC2dWorkflowConfig(WorkflowConfig):
    """
    infer-subc Provides access to structure workflow configuration
    """

    def __init__(self):
        self._all_functions = None
        self._available_workflow_names = None

    def get_available_workflows(self) -> List[str]:
        """
        Get the list of all workflows available through configuration
        """
        if self._available_workflow_names is None:
            json_list = sorted(InferSubC2dDirectories.get_structure_config_dir().glob("conf_*.json"))
            self._available_workflow_names = [p.stem[5:] for p in json_list]

        return self._available_workflow_names

    def get_all_functions(self) -> List[SegmenterFunction]:
        """
        Get the list of all available Functions from configuration
        """
        if self._all_functions is None:
            path = InferSubC2dDirectories.get_structure_config_dir() / "all_functions.json"

            try:
                with open(path) as file:
                    obj = json.load(file)
                    self._all_functions = self._all_functions_decoder(obj)
            except Exception as ex:
                raise ConfigurationException(f"Error reading json configuration from {path}") from ex

        return self._all_functions

    def get_workflow_definition(self, workflow_name: str) -> PrebuiltWorkflowDefinition:
        """
        Get a WorkflowDefinition for the given workflow from the corresponding
        prebuilt json structure config
        """
        if workflow_name is None or len(workflow_name.strip()) == 0:
            raise ValueError("workflow_name cannot be empty")

        if workflow_name not in self.get_available_workflows():
            raise ValueError(f"No workflow configuration available for {workflow_name}")

        path = InferSubC2dDirectories.get_structure_config_dir() / f"conf_{workflow_name}.json"

        return self.get_workflow_definition_from_config_file(path, workflow_name, prebuilt=True)


class InferSubC2dWorkflowEngine(WorkflowEngine):
    """
    infer-subc workflow engine
    Use this class to access and execute aicssegmentation structure workflows
    """

    def __init__(self, workflow_config: WorkflowConfig = None):
        self._workflow_config = workflow_config or InferSubC2dWorkflowConfig()
        self._workflow_definitions = self._load_workflow_definitions()
